# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# define useful functions
def dfheader_remove_upper_space(df):
    '''
    change column names of dataframe to lowercase with underscores
    '''
    # get list of column names
    oldcol = list(df.columns.values)
    # convert each column name to lowercase and replace space with underscore
    newcol = [s.lower().replace(' ', '_') for s in oldcol]
    # create a rename dict with the above
    rendict = dict(zip(oldcol, newcol))
    # return renamed dataframe
    return(df.rename(columns = rendict))
##end dfheader_remove_upper_space(df)

def dollar_format(df, col):
    '''
    Reformats columns with float values to strings with $x.yy format   
    '''
    return(df[col].map("${:,.2f}".format))
##end dollar_format(df, col)

def comma_format(df, col):
    '''
    Reformats columns with int values to strings with , to separate thousands   
    '''
    return(df[col].map("{:,}".format))
##end comma_format(df, col)


In [2]:
# File to Load
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School Data File and store into Pandas Dataframes and do some preprocessing to add more info
school_data = pd.read_csv(school_data_to_load)
school_data = dfheader_remove_upper_space(school_data)
school_data = school_data.rename(columns = {'size' : 'num_students', 'type' : 'school_type'})
school_data['student_budget'] = school_data.budget / school_data.num_students
school_data['total_students'] = comma_format(school_data, 'num_students')
school_data['total_budget'] = dollar_format(school_data, 'budget')
school_data['per_student_budget'] = dollar_format(school_data, 'student_budget')
# Create bins for per student spending budget
spending_bins = [0, 585.0, 615.0, 645.0, 675.0]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]
# add new column with spending range info into dataframe
school_data['spending_ranges'] = pd.cut(school_data.student_budget, spending_bins, labels=group_names)
# Create bins for number of students
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
# add new column with spending range info into dataframe
school_data['school_size'] = pd.cut(school_data.num_students, size_bins, labels=group_names)

# Read Student Data File and store into Pandas Dataframe
student_data = pd.read_csv(student_data_to_load)
student_data = dfheader_remove_upper_space(student_data)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
# add new columns that indicate if the student passed the subject i.e. scored >=70
school_data_complete['pass_math'] = school_data_complete.math_score//70
school_data_complete['pass_reading'] = school_data_complete.reading_score//70
#school_data_complete.head()

In [3]:
def total_num_schools(dfgrp):
    return dfgrp.school_id.nunique()
##end total_num_schools(dfgrp)

def total_students(dfgrp):
    return dfgrp.student_id.nunique()
##end total_students(dfgrp)

def total_budget(dfgrp):
    return dfgrp.budget.sum()
##end total_budget(dfgrp)

def avg_score(dfgrp, subcol):
    return dfgrp[subcol].mean()
##end avg_score(dfgrp, subcol)

def perc_pass(dfgrp, subcol):
    numpass = dfgrp[subcol].sum()
    retval = (numpass * 100.0) / total_students(dfgrp)
    return retval
##end perc_pass(dfgrp, subcol)

def summary(dfgrp, idx):
    # create the summary dataframe
    summary_df = pd.DataFrame({"total_students": total_students(dfgrp),
                               "avg_math_score": avg_score(dfgrp, 'math_score'),
                               "avg_reading_score": avg_score(dfgrp, 'reading_score'),
                               "perc_pass_math": perc_pass(dfgrp, 'pass_math'),
                               "perc_pass_reading": perc_pass(dfgrp, 'pass_reading')}, index=idx)
    # calculate overall passing percentage
    summary_df['perc_pass_overall'] = (summary_df.perc_pass_math + summary_df.perc_pass_reading)/2
    summary_df.total_students = comma_format(summary_df, 'total_students')

    return summary_df
##end summary(dfgrp, idx)

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [4]:
district_summary = summary(school_data_complete, [0])
district_summary.insert(loc=0, column='total_num_schools', value=total_num_schools(school_data))
district_summary.insert(loc=2, column='total_budget', value=total_budget(school_data))
district_summary.total_budget = dollar_format(district_summary, 'total_budget')
district_summary.head()

,total_num_schools,total_students,total_budget,avg_math_score,avg_reading_score,perc_pass_math,perc_pass_reading,perc_pass_overall
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,80.393158


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [5]:
# group by schools to generate summary for each school
grpcols = ['school_name', 'school_type', 'total_students', 'total_budget', 'per_student_budget']
school_grp = school_data_complete.groupby(grpcols)
school_summary = summary(school_grp, None)
del school_summary['total_students']

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [6]:
school_summary.sort_values(by='perc_pass_overall', ascending=False).head()

,,,,,avg_math_score,avg_reading_score,perc_pass_math,perc_pass_reading,perc_pass_overall
school_name,school_type,total_students,total_budget,per_student_budget,,,,,
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,95.203679


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [7]:
school_summary.sort_values(by='perc_pass_overall', ascending=True).head()

,,,,,avg_math_score,avg_reading_score,perc_pass_math,perc_pass_reading,perc_pass_overall
school_name,school_type,total_students,total_budget,per_student_budget,,,,,
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308


In [8]:
def grade_summary(df, subcol):
    # create group for the given school and grade level
    grade_grp = df.groupby(['school_name', 'grade'])
    # create the summary dataframe for the requested subject
    summary_df = pd.DataFrame({"avg_score": avg_score(grade_grp, subcol)})
    # unstack the multi-indexed dataframe with columns representing grades
    summary_df = summary_df.unstack(level='grade')
    summary_df.columns = summary_df.columns.droplevel()
    # reorder the columns to the correct order
    cols = summary_df.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    summary_df = summary_df[cols]
    
    return summary_df
##end grade_summary(df, subcol)

## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [9]:
grade_summary(school_data_complete, 'math_score')

grade,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [10]:
grade_summary(school_data_complete, 'reading_score')

grade,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [11]:
def avg_score_grp(dfgrp, subcol, lvl):
    return avg_score(dfgrp, subcol).mean(level=lvl)
##end avg_score_grp(dfgrp, subcol, lvl)

def perc_pass_grp(dfgrp, subcol, lvl):
    return perc_pass(dfgrp, subcol).mean(level=lvl)
##end avg_score_grp(dfgrp, subcol, lvl)

def type_summary(df, stype):
    # create group for the given school type for each school
    stype_school_grp = df.groupby([stype, 'school_name'])
    # create the summary dataframe
    summary_df = pd.DataFrame({"avg_math_score": avg_score_grp(stype_school_grp, 'math_score', stype),
                               "avg_reading_score": avg_score_grp(stype_school_grp, 'reading_score', stype),
                               "perc_pass_math": perc_pass_grp(stype_school_grp, 'pass_math', stype),
                               "perc_pass_reading": perc_pass_grp(stype_school_grp, 'pass_reading', stype)})
    # calculate overall passing percentage
    summary_df['perc_pass_overall'] = (summary_df.perc_pass_math + summary_df.perc_pass_reading)/2

    return summary_df
##end type_summary(df, stype)

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [12]:
type_summary(school_data_complete, 'spending_ranges')

,avg_math_score,avg_reading_score,perc_pass_math,perc_pass_reading,perc_pass_overall
spending_ranges,,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


## Scores by School Size

* Perform the same operations as above, based on school size.

In [13]:
type_summary(school_data_complete, 'school_size')

,avg_math_score,avg_reading_score,perc_pass_math,perc_pass_reading,perc_pass_overall
school_size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


## Scores by School Type

* Perform the same operations as above, based on school type.

In [14]:
type_summary(school_data_complete, 'school_type')

,avg_math_score,avg_reading_score,perc_pass_math,perc_pass_reading,perc_pass_overall
school_type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
